# DM HW1 110705013

## Import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np

## Preprocessing

Read input

In [ ]:
data = pd.read_csv('train.csv')
data.drop(columns=['Location', 'Date'], inplace=True)
columns = data.columns
data['ItemName'] = data['ItemName'].str.replace(' ', '')
for column in columns[1:]:
    data[column].replace(['#', '*', 'x', 'A'], [None, None, None, None], inplace=True)
    data[column] = data[column].apply(pd.to_numeric, errors='coerce')
item_amount = data.groupby('ItemName').groups.keys().__len__()
data = data.to_numpy()

Create new data column, row = items, hour & filling in nan values

In [ ]:
row_n, col_n = data.shape
item_names = data[0:item_amount, 0].reshape(-1)
dnew = {}
for name in item_names:
    dnew[name] = []

# insert into dictionary
for i in range(0, row_n, item_amount):
    for j in range(0, item_amount):
        dnew[item_names[j]].append(data[i + j, 1:])

# fill in nan values with the average of the previous and next value
for key in dnew.keys():
    dnew[key] = np.array(dnew[key]).flatten()
    for i in range(1,dnew[key].shape[0]-1):
        if np.isnan(dnew[key][i]):
            if not np.isnan(dnew[key][i - 1]) and not np.isnan(dnew[key][i + 1]):
                dnew[key][i] = (dnew[key][i- 1] + dnew[key][i+ 1]) / 2

# count the number of nan values in each column
count = np.zeros(item_amount)
for i, key in enumerate(dnew.keys()):
    for j in range(dnew[key].shape[0]):
        if np.isnan(dnew[key][j]):
            count[i] += 1
print('number of nans after filling:')
for i in range(item_amount):
    print(f'{item_names[i]}: {count[i]}', end = ', ')

data = pd.DataFrame.from_dict(dnew)
data = data.apply(pd.to_numeric, errors='coerce')
df = data
print("")
print(data.head())

Generate model input data

In [ ]:
# get the nn input data form and compute correlation
input_hours = 9
daily_data = data.to_numpy()
draw_data = []
for i in range(0, row_n - input_hours):
    temp = daily_data[i:i+input_hours,:].transpose().flatten()
    temp = temp.tolist()
    temp.append(daily_data[i + input_hours, 9])
    draw_data.append(temp)
labels = [[item+str(i) for i in range(input_hours)] for item in item_names]
labels = np.array(labels).flatten().tolist()
print(len(draw_data))
data = pd.DataFrame(draw_data, columns=labels + ['predict'])
print("Corr:")
corr = data.corr()
show = corr['predict'].to_numpy()

for i in range(0, len(show)-1):
    print(labels[i] + ' ', end='')
    print(show[i])

#### Visualize for anaylsis

Correlation coefficient matrix for hour data

In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f')
plt.show()

Correlation Coefficient matrix for model input data

In [ ]:
plt.figure(figsize=(80, 80))
sns.heatmap(corr, annot=True, fmt='.2f')
plt.savefig('correlation.png')
plt.show()

## Training

Extract specific columns for training

In [ ]:
# threshold = 0.35
# element amount = 24*240
input_hours = 9
item_names = [ 'CO', 'PM10', 'PM2.5'] #'AMB_TEMP', 'CH4',, 'NMHC', 'NO2', 'NOx',, 'THC'
filter_item = []
for item in item_names:
    if item == 'PM2.5' or item == 'PM10':
        for i in range(4, input_hours):
            filter_item.append(item + str(i))
    else:
        for i in range(6, input_hours):
            filter_item.append(item + str(i))
filter_item.append('predict')
new_data = data[filter_item]
print(new_data.head())
new_data = new_data.dropna()
dataset = new_data.to_numpy()
dataset = dataset.astype(np.float64)
row_n, col_n = dataset.shape

print(dataset.shape)

In [ ]:
new_data.head()

In [ ]:
np.random.seed(0)
np.random.shuffle(dataset)
trainingX = dataset[:int(0.8 * dataset.shape[0]), :-1]
trainingY = dataset[:int(0.8 * dataset.shape[0]), -1].reshape(-1, 1)
testingX = dataset[int(0.8 * dataset.shape[0]):, :-1]
testingY = dataset[int(0.8 * dataset.shape[0]):, -1].reshape(-1, 1)

In [ ]:
# do feature scaling with X^2
# min max scaling
# these does not help
trainingX = np.concatenate((trainingX, np.square(trainingX[:,:-3])), axis=1)
testingX = np.concatenate((testingX, np.square(testingX[:,:-3])), axis=1)
x_max = np.max(trainingX, axis=0)
x_min = np.min(trainingX, axis=0)
trainingX = (trainingX - x_min) / (x_max - x_min)
testingX = (testingX - x_min) / (x_max - x_min)

### Closed form solution

In [ ]:
def MSE(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)
def close_form(X, Y):
    X = np.hstack((X, np.ones((X.shape[0], 1))))
    return np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), Y)

def close_form_L2(X, Y, l):
    X = np.hstack((X, np.ones((X.shape[0], 1))))
    return np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + l * np.eye(X.shape[1])), X.T), Y)

weights = close_form_L2(trainingX, trainingY, 0.1)
predictedY = np.dot(np.hstack((testingX, np.ones((testingX.shape[0], 1)))), weights)
print('MSE:', MSE(testingY, predictedY))

plt.figure(figsize=(10, 10))
plt.plot(testingY, label='True', alpha=0.5)
plt.plot(predictedY, label='Predicted', alpha=0.5)
plt.legend()
plt.show()

In [ ]:
file_path = 'closed_form_weights'
print(weights.shape)
np.savez(file_path, weight=weights[:-1], intercept=np.array(weights[-1]), x_max=x_max, x_min=x_min)

### Gradient Descent

In [ ]:
class Linear():
    def __init__(self, input_size, output_size):
        self.W = np.random.randn(input_size, output_size)
        self.intercept = np.random.randn(output_size)
        self.input = None

    def forward(self, x):
        self.input = x
        return np.dot(x, self.W) + self.intercept
    
    def backward(self, gradient, lr):
        # L2 regularize
        gradient += 0.01 * np.dot(self.W.T, self.W).item()
        gradient_out = np.dot(gradient, self.W.T)
        self.W -= lr * np.dot(self.input.T, gradient)
        self.intercept -= lr * np.sum(gradient, axis=0)
        return gradient_out

class Adagrad():
    def __init__(self, lr=0.01):
        self.lr = lr
        self.G = None
    def getlr(self, gradient):
        if self.G is None:
            self.G = 1
        self.G += np.mean(gradient**2)/10
        return self.lr / np.sqrt(self.G + 1e-7)

In [ ]:
trainingX.shape

In [ ]:
from torch.utils.tensorboard import SummaryWriter
def train(lr, model):
    writer = SummaryWriter("lr_" + str(lr))
    np.random.seed(0)
    optimizer = Adagrad(lr=lr)
    epochs = 2000000
    for i in range(epochs):
        output = model.forward(trainingX)
        loss = (output - trainingY) ** 2
        gradient = 2 * (output - trainingY)
        #print(gradient.shape)
        lr = optimizer.getlr(gradient)
        gradient = model.backward(gradient, lr)
        if i % 1000 == 0:
            writer.add_scalar('Loss', loss.mean(), i)
            print('Epoch:', i, 'Loss:', loss.mean())
    np.savez('model.npy', weight=model.W, intercept=model.intercept)
    return -loss.mean()
        

In [ ]:
model = Linear(trainingX.shape[1], 1)
print(-train(0.7738, model))

## Draw result

In [ ]:
record = pd.read_csv('result.csv')
record = record.to_numpy()
data_size = 4103

plt.figure(figsize=(8, 8))
x_size = record.shape[0]
# x axis is record[:, 0]
# y axis is record[:, 1]
plt.xlabel('Data Size')
plt.ylabel('MSE')
plt.plot(data_size * record[:, 0], record[:, 1], label='True')

## Inference

In [ ]:
data = pd.read_csv('test.csv')
data.drop(columns=['date'], inplace=True)
columns = data.columns

data['ItemName'] = data['ItemName'].str.replace(' ', '')
print(data.head())
for column in columns[1:]:
    data[column].replace(['#', '*', 'x', 'A'], [None, None, None, None], inplace=True)
    data[column] = data[column].apply(pd.to_numeric, errors='coerce')
item_amount = data.groupby('ItemName').groups.keys().__len__()
data = data.to_numpy()

row_n, col_n = data.shape
item_names = data[0:item_amount, 0].reshape(-1)
dnew = {}
mean = {}
for name in item_names:
    dnew[name] = []
    

# insert into dictionary
for i in range(0, row_n, item_amount):
    for j in range(0, item_amount):
        dnew[item_names[j]].append(data[i + j, 1:].tolist())


for key in dnew.keys():
    dnew[key] = np.array(dnew[key]).flatten()
    mean[key] = np.nanmean(dnew[key])
    for i in range(1,dnew[key].shape[0]-1):
        if np.isnan(dnew[key][i]):
            if not np.isnan(dnew[key][i - 1]) and not np.isnan(dnew[key][i + 1]):
                dnew[key][i] = (dnew[key][i- 1] + dnew[key][i+ 1]) / 2

# count the number of nan values in each column
count = np.zeros(item_amount)
for i, key in enumerate(dnew.keys()):
    for j in range(dnew[key].shape[0]):
        if np.isnan(dnew[key][j]):
            count[i] += 1
            dnew[key][j] = mean[key]
print('number of nans after filling:')
for i in range(item_amount):
    print(f'{item_names[i]}: {count[i]}', end = ', ')
    
data = pd.DataFrame.from_dict(dnew)
data = data.apply(pd.to_numeric, errors='coerce')

# get the nn input data form and compute correlation
input_hours = 9
daily_data = data.to_numpy()
print(daily_data.shape)
draw_data = []
for i in range(0, daily_data.shape[0], input_hours):
    temp = daily_data[i:i+input_hours,:].transpose().flatten()
    temp = temp.tolist()
    #temp.append(daily_data[i + input_hours, 9])
    draw_data.append(temp)
print(len(draw_data))
labels = [[item+str(i) for i in range(input_hours)] for item in item_names]
labels = np.array(labels).flatten().tolist()
data = pd.DataFrame(draw_data, columns=labels)
print(data.size)
    
# threshold = 0.35
# element amount = 24*240
input_hours = 9
item_names = ['CO', 'PM10', 'PM2.5']# ['CO', 'NMHC', 'NO2', 'NOx', 'PM10', 'PM2.5', 'THC']
filter_item = []
for item in item_names:
    if item == 'PM2.5' or item == 'PM10':
        for i in range(4, input_hours):
            filter_item.append(item + str(i))
    else:
        for i in range(6, input_hours):
            filter_item.append(item + str(i))
new_data = data[filter_item]
print(new_data.size)
new_data = new_data.dropna()
print(new_data.head())
dataset = new_data.to_numpy()
dataset = dataset.astype(np.float64)
row_n, col_n = dataset.shape

print(dataset.shape)
    
model = Linear(dataset.shape[1]+3, 1)
loaded = np.load('closed_form_weights.npz')
model.W = loaded['weight']
model.intercept = loaded['intercept']
x_max = loaded['x_max']
x_min = loaded['x_min']

# do feature scaling with X^2
# min max scaling
# these does not help
dataset = np.concatenate((dataset, np.square(dataset[:,:-3])), axis=1)
dataset = (dataset - x_min) / (x_max - x_min)

# inference
predict = model.forward(dataset)
predict = predict.flatten()
print(predict.shape)

predict = predict.tolist()
out = [['index_'+str(i), predict[i]] for i in range(len(predict))]
df = pd.DataFrame(out, columns=['index', 'answer'])
df.to_csv('predict.csv', index=False)